<a href="https://colab.research.google.com/github/rudycav/Equities-Digital-Assets-Prediction-Model/blob/main/Project/%20model_predict.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [104]:
import yfinance as yf
import datetime
import pandas as pd
import numpy as np
from finta import TA
import matplotlib.pyplot as plt

from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import confusion_matrix, classification_report
from sklearn import metrics



In [107]:

def ticker_data(days, timeframe, ticker):
    start = (datetime.date.today() - datetime.timedelta(days))
    end = datetime.datetime.today()

    # Loading start, end time, and interval into YF package
    ticker_data = yf.download(ticker, start=start, end=end, interval=timeframe)
    ticker_data.rename(columns={"Close": 'close', "High": 'high', "Low": 'low', 
                                'Volume': 'volume', 'Open': 'open'}, inplace=True)

    return ticker_data.head()
    

data = ticker_data(10, '1d', 'SPY')
data


[*********************100%***********************]  1 of 1 completed


,open,high,low,close,Adj Close,volume
Date,,,,,,
2022-06-30,376.239990,380.660004,372.559998,377.250000,377.250000,112508300
2022-07-01,376.559998,381.700012,373.799988,381.239990,381.239990,74792100
2022-07-05,375.880005,381.980011,372.899994,381.959991,381.959991,81438000
2022-07-06,382.109985,385.869995,379.600006,383.250000,383.250000,70426200
2022-07-07,385.119995,389.829987,383.269989,388.989990,388.989990,64525900


In [149]:
def exponential_weight(data, alpha):
  # Exponential weight collects more accurate float integers
  return data.ewm(alpha=alpha).mean()

exp_weight = exponential_weight(data, 0.65)

exp_weight.head()

,open,high,low,close,Adj Close,volume
Date,,,,,,
2022-06-30,376.239990,380.660004,372.559998,377.250000,377.250000,1.125083e+08
2022-07-01,376.477033,381.430380,373.478509,380.205548,380.205548,8.457037e+07
2022-07-05,376.071581,381.803644,373.085629,381.397021,381.397021,8.244313e+07
2022-07-06,380.056340,384.487040,377.384484,382.619807,382.619807,7.451312e+07
2022-07-07,383.365094,387.978292,381.230261,386.782288,386.782288,6.798715e+07


In [152]:
def indicators(data):

  ta_indicators = ['RSI', 'MACD']

  data['EMA200'] = data['close'] / data['close'].ewm(200).mean()
  data['EMA50'] = data['close'] / data['close'].ewm(50).mean()
  data['EMA20'] = data['close'] / data['close'].ewm(20).mean()
  data['EMA9'] = data['close'] / data['close'].ewm(9).mean()

  for indicator in ta_indicators:
    combine_indicators = eval('TA.' + indicator + '(exp_weight)')
    #if not isinstance(ind_data, pd.DataFrame):
        #ind_data = ind_data.to_frame()
    data = data.merge(combine_indicators, left_index=True, right_index=True)
    data.rename(columns={"14 period RSI": 'RSI'}, inplace=True)
  
  data.drop(['open', 'high', 'low', 'Adj Close', 'volume'], axis=1, inplace=True)

  return data

indicator_data = indicators(exp_weight)
indicator_data

,close,EMA200,EMA50,EMA20,EMA9,RSI,MACD,SIGNAL
Date,,,,,,,,
2022-06-30,377.250000,1.000000,1.000000,1.000000,1.000000,NaN,0.000000,0.000000
2022-07-01,380.205548,1.003892,1.003863,1.003806,1.003696,100.0,0.066310,0.036839
2022-07-05,381.397021,1.004669,1.004615,1.004510,1.004306,100.0,0.121440,0.071512
2022-07-06,382.619807,1.005891,1.005807,1.005642,1.005326,100.0,0.187686,0.110866
2022-07-07,386.782288,1.013388,1.013219,1.012888,1.012243,100.0,0.385367,0.192524


In [157]:
live_pred_data = indicator_data.iloc[0:6]